# Rappresentazione intermedia

Nel notebook precedente abbiamo creato una specie di calcolatrice, con la possibilità di utilizzare assegnamenti e variabili per la memorizzazione di risultati intermedi. Questo esercizio ci è servito per familiarizzare con le grammatiche libere e con lo stumento ply.yacc per la generazione di un parser.

In questo notebook facciamo il passo successivo, ultimo passo che prenderemo in analisi del processo di interpretazione di un linguaggio. Il passo che faremo oggi è trasformare l'applicazione operando la separazione fra compilatore e interprete.

Prenderemo come esempio il codice ```expressions``` dello scorso notebook ma con qualche aggiunta, tipo la possibilità di eseguire più linee di codice alla volta e la possibilità di utilizzare un comando di stampa. Il codice del notebook corrente lo troviamo nella catella ```language``` sempre in ```codes```.

Procediamo quindi arricchendo la nostra grammatica:

- $\mathcal{P} \rightarrow L$
- $L \rightarrow L; S | S$
- $S \rightarrow A | E | print(E) | \varepsilon$

Le seguenti produzioni ci permettono di specificare un programma come un insieme vuoto, quindi nessun comando, oppure come una sequenza di un numero arbitrario di comandi separati da ```;```. Inoltre abbiamo introdotto anche la possibilità di stampa.

Avremo quindi la seguente grammatica libera completa:

- $\mathcal{P} \rightarrow L$
- $L \rightarrow L; S | S$
- $S \rightarrow A | E | print(E) | \varepsilon$
- $A \rightarrow id = E$
- $E \rightarrow E + T | E - T | T$
- $T \rightarrow T \times P | T / P | P$
- $P \rightarrow F ^ P | F | -F$
- $F \rightarrow id | pi | sqrt(E) | n | (E)$

> Per i più curiosi:  
Per avere un'idea della semplicità, potete confrontare la grammatica completa in [Python](https://docs.python.org/3/reference/grammar.html) con la grammatica completa in [C](https://www.lysator.liu.se/c/ANSI-C-grammar-y.html)

## Rappresentazione intermedia

Il processo di rappresentazione intermedia si tratta sostanzialmente di applicare un algoritmo. Bisogna infatti definire la rappresentazione intermedia ad albero, ovvero l'AST che abbiamo precedentemente visto, e definire un algoritmo di valutazione sulla base di una vista post ordine dello stesso albero.

La rappresentazione utilizza le tuple. Infatti l'albero lo rappresenteremo con la nozione delle tuple dove:

- Il primo elemento della tupla, l'indice 0, è un intero il quale denota un opcode, ovvero un codice operativo condiviso tra parser e interprete. L'opcode generalmente è arbitrario, l'importante che sia condiviso dalle parti in gioco
- Gli elementi di indice successivo allo 0 sono a loro volta tuple che rappresentano sottoalberi oppure numeri o identificatori

Possiamo fare un esempio banale per comprendere il processo. Immaginiamo di avere questo frammento di codice.

```
a = sqrt(3);
print(a);
print(5 * a)
```

Supponiamo i sequenti codici operativi, opcode. Ricordo che gli opcode sono arbitrari e non hanno quindi una connessione logica.

- 1 = NUM
- 2 = ID
- 3 = SEQ
- 4 = ASSIGN
- 8 = MUL
- 20 = PRINT
- 30 = SQRT

La prima cosa che fa il parser, come già noto, è di utilizzare le produzioni della grammatica per arrivare ad avere la sequente frase linguistica:

$$ID = SQRT(n); \ PRINT(ID); \ PRINT(n \times ID)$$

Successivamente il parser utilizza le riduzioni per generare l'AST, il quale figura come segue.

![Image 10](images/image-10.png)

Dove i nodi intermedi rappresentano gli opcode, mentre le foglie i valori rispettivi a identificatori o valori numerici.

La rappresentazione dell'AST sotto forma di tupla è la seguente.

$$(3, (3, (4, a, (30, (1, 3))), (20, (2, a))), (20, (8, (1, 5), (2, a))))$$

## Regole per la costruzione dell'AST

Il passaggio che dobbiamo fare adesso è fare in modo che il parser crei l'albero sotto forma di tupla come abbiamo appena visto, in modo che l'interprete riesca ad possa eseguire il nostro programma.

Questo passaggio è molto diretto. Prendiamo come esempio una produzione qualsiasi, tipo $L \rightarrow L; S$. Nel momento in cui viene applicata la riduzione, le variabili associate a $L$ e $S$ memorizzano già delle tuple corrispondenti ai relativi comandi. Ne segue quindi che la riduzione dovrà costruire una tupla in cui l'opcode è, in questo esempio, ```SEQ```, mentre gli elementi restanti sono proprio le tuple in questione, quelle di $L$ e di $S$.

In [ ]:
def p_statement_list(p):
    'statement_list : statement_list SEP statement'
    p[0] = (SEQ, p[1], p[3])

Procediamo in questo modo per tutte le produzioni e arriveremo ad avere il parser in grado di restituire un AST.

## Il formato su disco

Il nostro parser dovrà in qualche modo salvare i suoi risultati, ovvero l'AST, da qualche parte. Noi procederemo salvando le informazioni in un file JSON permettendoci la lettura in modo agevole.

```python
import json

# Scrittura
jsonfile = json.dumps(ast)

...

# Lettura
try:
     ast = json.loads(open(sys.argv[1]).read())
except JSONDecodeError:
     # Eventuale recupero
```

Facendo in questo modo, l'interprete potrà leggere il file ```ast.json``` prodotto dal parser per interpretare il nostro programma.

## L'interprete

Arrivati a questo punto, il gioco è fatto. Ci basta scrivere un codice che permetta di interpretare il nostro programma. Per fare ciò, utilizzeremo il modulo ```operator``` che contiene gli operatori definiti come funzioni.

L'interprete quindi visita l'AST salvato nel disco ed esegue gli opcode dopo aver ricorsivamente valutato tutti i sotto alberi in post-ordine.

Tutti i file scritti in questo notebook sono presnti in ```codes/language```. In questa directory troviamo lo scanner scritto inizialmente per ottenere i token a noi necessari, troviamo il parser che ci permette di scrivere nel disco il relativo AST a partire dai token dello scanner e per finire troveremo anche l'interprete, che altro non fa che interpretare il nostro programma, scritto nel file ```program.lang```. L'estensione ```.lang``` del nostro programma non ha alcun effetto sul funzionamento dell'interprete, potevamo addirittura omettere l'estensione, UNIX non è basato sull'estensione del file.

Concludiamo questo notebook con un accenno sul come utilizzare questi file.

Per eseguire il nostro programma custom abbiamo bisogno di fare due passaggi:

1. Utilizzare il parser per generare nel disco il file contenente l'AST
2. Utilizzare l'interprete per appunto interpretare il nostro programma

In [3]:
!python codes/language/parser.py -h

Uso: ./parser.py [opzioni] -p input_prog | -f prog_file
OPTIONS: 
	-h: stampa questo help e termina
	-t: stampa su stdout l'Abstract Syntax Tree


In [4]:
!python codes/language/parser.py -t -f codes/language/program.lang

Abstract Syntax Tree internal representation
(3,
 (3, (4, 'a', (30, (1, 3))),
  (20, (2, 'a'))),
 (20, (8, (1, 5), (2, 'a'))))


In [5]:
!python codes/language/interpreter.py codes/language/ast.json

1.7320508075688772
8.660254037844386


Sarebbe stato possibile fare tutti e due i passaggi in uno solo, lasciando al nostro codice tutto il lavoro, ma facendo in questo modo risultano immediati i compiti singoli di frontend, quindi scanner e parser, e di backend, ovvero l'interprete.